# Corrective RAG (CRAG)

## Setting Up the Environment

In [22]:
import numpy as np
import json
import fitz  # PyMuPDF
import requests
from typing import List, Dict, Tuple, Any
import re
from urllib.parse import quote_plus
import time

In [23]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Document Processing Functions

In [24]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 전체 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일 경로

    Returns:
        str: 추출된 전체 텍스트
    """
    print(f"{pdf_path}에서 텍스트 추출 중...")

    # PDF 파일 열기
    pdf = fitz.open(pdf_path)
    text = ""

    # 각 페이지를 순회하며 텍스트를 이어 붙임
    for page_num in range(len(pdf)):
        page = pdf[page_num]
        text += page.get_text()  # 현재 페이지에서 텍스트 추출 및 누적

    return text  # 전체 텍스트 반환

In [25]:
def chunk_text(text, chunk_size=1000, overlap=200):
    """
    효율적인 검색과 처리를 위한 텍스트 청크 분할 함수입니다.

    이 함수는 입력 텍스트를 지정된 길이의 청크로 나누되,
    각 청크 간에 일정한 문자 수만큼 중첩(overlap)을 유지합니다.
    RAG 시스템에서는 문맥을 유지한 정확한 검색을 위해 청크 분할이 매우 중요합니다.

    Args:
        text (str): 분할할 원본 텍스트
        chunk_size (int): 각 청크의 최대 문자 수
        overlap (int): 청크 간 중첩될 문자 수 (문맥 연속성 확보용)

    Returns:
        List[Dict]: 청크 목록. 각 청크는 다음 필드를 포함합니다:
            - text: 실제 청크 텍스트
            - metadata: 위치 정보와 출처 타입이 포함된 딕셔너리
    """
    chunks = []

    # 슬라이딩 윈도우 방식으로 청크 생성
    # (chunk_size - overlap) 단위로 이동하며 중첩을 유지
    for i in range(0, len(text), chunk_size - overlap):
        chunk_text = text[i:i + chunk_size]  # 현재 구간의 텍스트 추출

        if chunk_text:  # 빈 청크는 제외
            chunks.append({
                "text": chunk_text,
                "metadata": {
                    "start_pos": i,  # 원문에서의 시작 위치
                    "end_pos": i + len(chunk_text),  # 종료 위치
                    "source_type": "document"  # 문서 기반 청크임을 명시
                }
            })

    print(f"총 {len(chunks)}개의 텍스트 청크가 생성되었습니다.")
    return chunks

## Simple Vector Store Implementation

In [26]:
class SimpleVectorStore:
    """
    NumPy를 활용한 간단한 벡터 저장소 구현 클래스
    """
    def __init__(self):
        # 임베딩 벡터, 텍스트, 메타데이터를 저장할 리스트 초기화
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 하나의 항목을 추가합니다.
        
        Args:
            text (str): 원본 텍스트
            embedding (List[float]): 임베딩 벡터
            metadata (Dict, optional): 부가 메타데이터 (선택)
        """
        # 각각의 리스트에 항목 추가
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def add_items(self, items, embeddings):
        """
        여러 개의 항목을 한 번에 저장소에 추가합니다.
        
        Args:
            items (List[Dict]): 텍스트 및 메타데이터를 포함한 항목 리스트
            embeddings (List[List[float]]): 임베딩 벡터 리스트
        """
        # 각 항목과 임베딩을 쌍으로 반복하여 추가
        for i, (item, embedding) in enumerate(zip(items, embeddings)):
            self.add_item(
                text=item["text"],
                embedding=embedding,
                metadata=item.get("metadata", {})
            )
    
    def similarity_search(self, query_embedding, k=5):
        """
        주어진 쿼리 임베딩과 가장 유사한 항목들을 검색합니다.
        
        Args:
            query_embedding (List[float]): 쿼리 임베딩 벡터
            k (int): 반환할 상위 유사 항목 수
            
        Returns:
            List[Dict]: 유사도가 높은 상위 k개 항목
        """
        # 저장소가 비어 있을 경우 빈 리스트 반환
        if not self.vectors:
            return []
        
        # 쿼리 벡터를 numpy 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 계산하여 저장
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 유사도 기준으로 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 항목 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": float(score)
            })
        
        return results

## Creating Embeddings

In [27]:
def create_embeddings(texts, model="text-embedding-3-small"):
    """
    OpenAI 임베딩 모델을 사용하여 텍스트의 벡터 임베딩을 생성합니다.

    임베딩은 텍스트의 의미적 유사성을 수치화한 벡터 표현으로,
    RAG 시스템에서는 질문과 문서 청크 간의 유사도를 비교하는 데 핵심적으로 활용됩니다.

    Args:
        texts (str 또는 List[str]): 임베딩할 텍스트. 문자열 하나 또는 문자열 리스트 가능
        model (str): 사용할 임베딩 모델 이름. 기본값은 "text-embedding-3-small"

    Returns:
        List[List[float]] 또는 List[float]:
            - 입력이 리스트일 경우: 임베딩 벡터 리스트 반환
            - 입력이 단일 문자열일 경우: 하나의 임베딩 벡터 반환
    """
    # 단일 문자열 입력도 리스트로 처리
    input_texts = texts if isinstance(texts, list) else [texts]

    # 배치 단위로 처리하여 API 제한(속도/페이로드) 회피
    batch_size = 100
    all_embeddings = []

    # 배치 단위로 임베딩 생성
    for i in range(0, len(input_texts), batch_size):
        batch = input_texts[i:i + batch_size]

        # 현재 배치에 대해 OpenAI 임베딩 API 호출
        response = client.embeddings.create(
            model=model,
            input=batch
        )

        # 응답에서 임베딩 벡터 추출
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)

    # 입력이 문자열 하나였다면 첫 번째 벡터만 반환
    if isinstance(texts, str):
        return all_embeddings[0]

    # 리스트 입력이라면 전체 임베딩 리스트 반환
    return all_embeddings

## Document Processing Pipeline

In [28]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    PDF 문서를 벡터 저장소(Vector Store)로 처리합니다.

    Args:
        pdf_path (str): PDF 파일 경로
        chunk_size (int): 각 청크의 최대 길이 (문자 수 기준)
        chunk_overlap (int): 청크 간 중첩되는 문자 수

    Returns:
        SimpleVectorStore: 문서 청크와 임베딩이 포함된 벡터 저장소 객체
    """
    # 1단계: PDF 파일에서 텍스트 추출
    text = extract_text_from_pdf(pdf_path)

    # 2단계: 텍스트를 중첩된 청크로 분할
    chunks = chunk_text(text, chunk_size, chunk_overlap)

    # 3단계: 각 청크에 대한 임베딩 생성
    print("청크 임베딩 생성 중...")
    chunk_texts = [chunk["text"] for chunk in chunks]
    chunk_embeddings = create_embeddings(chunk_texts)

    # 4단계: 벡터 저장소 초기화
    vector_store = SimpleVectorStore()

    # 5단계: 청크와 임베딩을 저장소에 추가
    vector_store.add_items(chunks, chunk_embeddings)

    print(f"{len(chunks)}개의 청크로 구성된 벡터 저장소가 생성되었습니다.")
    return vector_store

## Relevance Evaluation Function

In [29]:
def evaluate_document_relevance(query, document):
    """
    문서가 쿼리와 얼마나 관련 있는지를 평가합니다.

    Args:
        query (str): 사용자 질문
        document (str): 평가할 문서 텍스트

    Returns:
        float: 관련도 점수 (0~1 사이의 값)
    """
    # 시스템 프롬프트: 모델에게 작업 방식 지시
    system_prompt = """
    당신은 문서의 관련성을 평가하는 전문가입니다.
    아래의 쿼리와 문서를 기준으로 관련성을 0에서 1 사이의 점수로 평가하세요.
    0은 완전히 무관함을, 1은 완전히 관련됨을 의미합니다.
    결과는 숫자(float) 하나만 출력하세요.
    """

    # 사용자 프롬프트 구성
    user_prompt = f"질문: {query}\n\n문서:\n{document}"

    try:
        # OpenAI API 호출로 관련성 평가 요청
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 사용할 모델
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,     # 변동 없는 일관된 응답 유도
            max_tokens=5       # 숫자 하나만 받으면 되므로 최소 토큰
        )

        # 응답에서 점수 텍스트 추출
        score_text = response.choices[0].message.content.strip()

        # 정규표현식으로 float 값 추출
        score_match = re.search(r'(\d+(\.\d+)?)', score_text)
        if score_match:
            return float(score_match.group(1))  # 점수를 float으로 반환
        return 0.5  # 파싱 실패 시 중간값 반환

    except Exception as e:
        # 에러 발생 시 기본값 반환
        print(f"문서 관련성 평가 오류: {e}")
        return 0.5

## Web Search Function

In [30]:
def duck_duck_go_search(query, num_results=3):
    """
    DuckDuckGo를 사용하여 웹 검색을 수행합니다.

    Args:
        query (str): 검색할 질의 문자열
        num_results (int): 반환할 최대 결과 수

    Returns:
        Tuple[str, List[Dict]]: 
            - results_text: 요약된 검색 결과 텍스트
            - sources: 각 결과의 출처 메타데이터 (제목 및 URL 포함)
    """
    # 쿼리를 URL에 사용할 수 있도록 인코딩
    encoded_query = quote_plus(query)

    # DuckDuckGo API 엔드포인트 (비공식 JSON API)
    url = f"https://api.duckduckgo.com/?q={encoded_query}&format=json"

    try:
        # API 요청 수행
        response = requests.get(url, headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        })
        data = response.json()

        results_text = ""
        sources = []

        # 요약(abstract)이 있을 경우 추가
        if data.get("AbstractText"):
            results_text += f"{data['AbstractText']}\n\n"
            sources.append({
                "title": data.get("AbstractSource", "Wikipedia"),
                "url": data.get("AbstractURL", "")
            })

        # 관련 주제(RelatedTopics)에서 추가 결과 수집
        for topic in data.get("RelatedTopics", [])[:num_results]:
            if "Text" in topic and "FirstURL" in topic:
                results_text += f"{topic['Text']}\n\n"
                sources.append({
                    "title": topic.get("Text", "").split(" - ")[0],
                    "url": topic.get("FirstURL", "")
                })

        return results_text, sources

    except Exception as e:
        # 메인 검색 실패 시 예외 출력
        print(f"웹 검색 중 오류 발생: {e}")

        # 백업 API 시도 (SerpAPI 사용)
        try:
            backup_url = f"https://serpapi.com/search.json?q={encoded_query}&engine=duckduckgo"
            response = requests.get(backup_url)
            data = response.json()

            results_text = ""
            sources = []

            # 백업 API에서 유기적 검색 결과 수집
            for result in data.get("organic_results", [])[:num_results]:
                results_text += f"{result.get('title', '')}: {result.get('snippet', '')}\n\n"
                sources.append({
                    "title": result.get("title", ""),
                    "url": result.get("link", "")
                })

            return results_text, sources

        except Exception as backup_error:
            # 백업 API도 실패할 경우 기본 메시지 반환
            print(f"백업 검색도 실패했습니다: {backup_error}")
            return "검색 결과를 가져오지 못했습니다.", []

In [31]:
def rewrite_search_query(query):
    """
    사용자의 원래 질문을 웹 검색에 더 적합하도록 재작성합니다.

    Args:
        query (str): 원본 사용자 질문

    Returns:
        str: 웹 검색 최적화된 재작성 쿼리
    """
    # LLM에게 검색 친화적 쿼리를 생성하도록 지시하는 시스템 프롬프트
    system_prompt = """
    당신은 효과적인 검색 쿼리를 만드는 전문가입니다.
    주어진 질문을 웹 검색 엔진에 적합하도록 재작성하세요.
    핵심 키워드와 사실 중심으로 표현하고, 불필요한 단어는 제거하며, 간결하게 만드세요.
    """

    try:
        # OpenAI API를 호출하여 쿼리를 재작성
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 사용할 모델 지정
            messages=[
                {"role": "system", "content": system_prompt},  # 시스템 지시
                {"role": "user", "content": f"원래 질문: {query}\n\n재작성된 검색 쿼리:"}
            ],
            temperature=0.3,  # 적당한 창의성 유지
            max_tokens=50  # 너무 길지 않게 제한
        )

        # 재작성된 쿼리 텍스트 반환
        return response.choices[0].message.content.strip()

    except Exception as e:
        # 예외 발생 시 원래 쿼리 반환
        print(f"검색 쿼리 재작성 중 오류 발생: {e}")
        return query

In [32]:
def perform_web_search(query):
    """
    쿼리를 웹 검색에 적합하게 재작성한 뒤, 웹 검색을 수행합니다.

    Args:
        query (str): 사용자의 원래 질문

    Returns:
        Tuple[str, List[Dict]]: 
            - 검색 결과 텍스트 (요약된 결과들)
            - 출처 메타데이터 리스트 (각 항목에 title과 url 포함)
    """
    # 1단계: 검색 성능 향상을 위해 쿼리를 재작성
    rewritten_query = rewrite_search_query(query)
    print(f"재작성된 검색 쿼리: {rewritten_query}")

    # 2단계: DuckDuckGo를 사용하여 웹 검색 실행
    results_text, sources = duck_duck_go_search(rewritten_query)

    # 3단계: 결과 텍스트와 출처 정보 반환
    return results_text, sources

## Knowledge Refinement Function

In [33]:
def refine_knowledge(text):
    """
    텍스트에서 핵심 정보를 추출하고 정제된 요약 형태로 반환합니다.

    Args:
        text (str): 요약 및 정제할 원본 텍스트

    Returns:
        str: 정제된 핵심 정보 (• 로 시작하는 글머리표 리스트)
    """
    # LLM에게 명확한 글머리표 형식으로 핵심 정보를 추출하도록 지시
    system_prompt = """
    아래 텍스트에서 가장 핵심적인 정보를 간결하고 명확한 글머리표 형태로 정리하세요.
    가장 중요한 사실과 관련된 내용을 중심으로 요약하되,
    각 항목은 새 줄에서 "• "로 시작하는 글머리표 형식으로 작성하세요.
    """

    try:
        # OpenAI API 호출하여 텍스트 정제 요청
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"정제할 텍스트:\n\n{text}"}
            ],
            temperature=0.3  # 핵심 유지에 적절한 온도 설정
        )

        # 정제된 요약 내용 반환
        return response.choices[0].message.content.strip()

    except Exception as e:
        # 오류 발생 시 원본 텍스트 반환
        print(f"지식 정제 중 오류 발생: {e}")
        return text

## Core CRAG Process

In [34]:
def crag_process(query, vector_store, k=3):
    """
    CRAG(Corrective RAG) 프로세스를 실행합니다.

    Args:
        query (str): 사용자 질문
        vector_store (SimpleVectorStore): 문서 청크를 포함한 벡터 저장소
        k (int): 초기 검색할 문서 수

    Returns:
        Dict: 최종 응답과 디버깅 정보를 포함한 처리 결과
    """
    print(f"\n=== CRAG 방식으로 쿼리 처리 중: {query} ===\n")

    # 1단계: 쿼리 임베딩 생성 및 유사 문서 검색
    print("초기 문서 검색 중...")
    query_embedding = create_embeddings(query)
    retrieved_docs = vector_store.similarity_search(query_embedding, k=k)

    # 2단계: 각 문서에 대해 관련도 평가
    print("문서 관련도 평가 중...")
    relevance_scores = []
    for doc in retrieved_docs:
        score = evaluate_document_relevance(query, doc["text"])
        relevance_scores.append(score)
        doc["relevance"] = score
        print(f"문서 관련도 점수: {score:.2f}")

    # 3단계: 최고 관련도 점수를 기준으로 처리 전략 결정
    max_score = max(relevance_scores) if relevance_scores else 0
    best_doc_idx = relevance_scores.index(max_score) if relevance_scores else -1

    sources = []          # 출처 정보 리스트
    final_knowledge = ""  # 응답 생성을 위한 최종 정보 컨텐츠

    # 4단계: 관련도에 따라 지식 수집 전략 분기 처리
    if max_score > 0.7:
        # Case 1: 높은 관련도 - 문서만 사용
        print(f"높은 관련도 ({max_score:.2f}) - 문서 직접 활용")
        best_doc = retrieved_docs[best_doc_idx]["text"]
        final_knowledge = best_doc
        sources.append({
            "title": "내부 문서",
            "url": ""
        })

    elif max_score < 0.3:
        # Case 2: 낮은 관련도 - 웹 검색 수행
        print(f"낮은 관련도 ({max_score:.2f}) - 웹 검색 수행")
        web_results, web_sources = perform_web_search(query)
        final_knowledge = refine_knowledge(web_results)
        sources.extend(web_sources)

    else:
        # Case 3: 중간 관련도 - 내부 문서 + 웹 검색 결합
        print(f"중간 관련도 ({max_score:.2f}) - 문서 + 웹 검색 병합")
        best_doc = retrieved_docs[best_doc_idx]["text"]
        refined_doc = refine_knowledge(best_doc)

        web_results, web_sources = perform_web_search(query)
        refined_web = refine_knowledge(web_results)

        final_knowledge = f"내부 문서 발췌:\n{refined_doc}\n\n웹 검색 요약:\n{refined_web}"
        sources.append({
            "title": "내부 문서",
            "url": ""
        })
        sources.extend(web_sources)

    # 5단계: 최종 응답 생성
    print("최종 응답 생성 중...")
    response = generate_response(query, final_knowledge, sources)

    # 최종 결과 반환
    return {
        "query": query,
        "response": response,
        "retrieved_docs": retrieved_docs,
        "relevance_scores": relevance_scores,
        "max_relevance": max_score,
        "final_knowledge": final_knowledge,
        "sources": sources
    }

## Response Generation

In [35]:
def generate_response(query, knowledge, sources):
    """
    주어진 질문과 지식을 바탕으로 AI 응답을 생성합니다.

    Args:
        query (str): 사용자 질문
        knowledge (str): 응답의 근거가 될 지식 내용
        sources (List[Dict]): 출처 정보 리스트 (각 항목은 title 및 url 포함)

    Returns:
        str: 생성된 AI 응답
    """
    # 출처 정보를 프롬프트에 맞게 포맷팅
    sources_text = ""
    for source in sources:
        title = source.get("title", "출처 없음")
        url = source.get("url", "")
        if url:
            sources_text += f"- {title}: {url}\n"
        else:
            sources_text += f"- {title}\n"

    # 시스템 프롬프트: 모델에게 응답 작성 기준 지시
    system_prompt = """
    당신은 유용하고 신뢰할 수 있는 AI 어시스턴트입니다.
    아래 지식을 기반으로 명확하고 포괄적인 답변을 작성하세요.
    질문에 완전히 답할 수 없다면 그 한계를 명확히 언급하세요.
    응답 마지막에 출처 목록을 포함하세요.
    """

    # 사용자 프롬프트: 질문 + 지식 + 출처를 포함
    user_prompt = f"""
    질문: {query}

    지식:
    {knowledge}

    출처:
    {sources_text}

    위 정보를 바탕으로 질문에 대해 유익한 응답을 작성해 주세요.
    응답 마지막에 위 출처들을 함께 포함해 주세요.
    """

    try:
        # OpenAI API 호출을 통해 응답 생성
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 고품질 응답 생성을 위한 GPT-4 사용
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2  # 일관된 톤을 위한 낮은 창의성
        )

        # 생성된 응답 반환
        return response.choices[0].message.content.strip()

    except Exception as e:
        # 오류 발생 시 예외 메시지와 함께 사과 응답 반환
        print(f"응답 생성 중 오류 발생: {e}")
        return f"죄송합니다. 질문 '{query}'에 대한 응답 생성 중 오류가 발생했습니다. 오류 내용: {str(e)}"

## Evaluation Functions

In [36]:
def evaluate_crag_response(query, response, reference_answer=None):
    """
    CRAG 응답의 품질을 평가합니다.

    Args:
        query (str): 사용자 질문
        response (str): 생성된 AI 응답
        reference_answer (str, optional): 기준이 되는 정답 (비교용, 선택)

    Returns:
        Dict: 평가 기준별 점수와 설명, 종합 점수를 포함한 평가 결과
    """
    # 시스템 프롬프트: 평가 기준 안내
    system_prompt = """
    당신은 AI 응답 품질 평가 전문가입니다.
    아래의 기준에 따라 주어진 응답을 평가하세요:

    1. 관련성 (0~10): 응답이 질문에 얼마나 직접적으로 답하고 있는가?
    2. 정확성 (0~10): 정보가 사실에 얼마나 부합하는가?
    3. 완전성 (0~10): 질문의 모든 측면을 얼마나 충실히 다루는가?
    4. 명확성 (0~10): 응답이 얼마나 명확하고 이해하기 쉬운가?
    5. 출처 품질 (0~10): 응답이 얼마나 적절하게 출처를 인용하고 있는가?

    결과는 다음 항목을 포함하는 JSON 형식으로 반환하세요:
    - 각 기준별 점수 및 간단한 평가 설명
    - 종합 점수(overall_score, 0~10)
    - 전체 평가 요약(summary)
    """

    # 사용자 프롬프트 구성
    user_prompt = f"""
    질문: {query}

    평가 대상 응답:
    {response}
    """

    # 기준 정답이 제공된 경우 포함
    if reference_answer:
        user_prompt += f"""
    기준 정답 (비교용):
    {reference_answer}
    """

    try:
        # GPT-4 모델을 사용해 평가 요청
        evaluation_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"},
            temperature=0
        )

        # 평가 결과를 JSON으로 파싱
        evaluation = json.loads(evaluation_response.choices[0].message.content)
        return evaluation

    except Exception as e:
        # 예외 발생 시 기본값 반환
        print(f"응답 평가 중 오류 발생: {e}")
        return {
            "error": str(e),
            "overall_score": 0,
            "summary": "오류로 인해 평가에 실패했습니다."
        }

In [37]:
def compare_crag_vs_standard_rag(query, vector_store, reference_answer=None):
    """
    동일한 쿼리에 대해 CRAG 방식과 일반 RAG 방식을 비교합니다.

    Args:
        query (str): 사용자 질문
        vector_store (SimpleVectorStore): 문서 청크가 저장된 벡터 저장소
        reference_answer (str, optional): 비교용 기준 정답 (선택)

    Returns:
        Dict: 두 방식의 응답, 평가 점수, 비교 결과를 포함한 정보
    """
    # 1단계: CRAG 방식 실행
    print("\n***CRAG 실행 중***")
    crag_result = crag_process(query, vector_store)
    crag_response = crag_result["response"]

    # 2단계: 일반 RAG 방식 실행 (단순 유사 문서 검색 후 응답 생성)
    print("\n***일반 RAG 실행 중***")
    query_embedding = create_embeddings(query)
    retrieved_docs = vector_store.similarity_search(query_embedding, k=3)
    combined_text = "\n\n".join([doc["text"] for doc in retrieved_docs])
    standard_sources = [{"title": "문서", "url": ""}]
    standard_response = generate_response(query, combined_text, standard_sources)

    # 3단계: CRAG 응답 평가
    print("\n***CRAG 응답 평가 중***")
    crag_eval = evaluate_crag_response(query, crag_response, reference_answer)

    # 4단계: 일반 RAG 응답 평가
    print("\n***일반 RAG 응답 평가 중***")
    standard_eval = evaluate_crag_response(query, standard_response, reference_answer)

    # 5단계: 두 응답 비교
    print("\n***두 방식 응답 비교 중***")
    comparison = compare_responses(query, crag_response, standard_response, reference_answer)

    # 결과 딕셔너리 반환
    return {
        "query": query,
        "crag_response": crag_response,
        "standard_response": standard_response,
        "reference_answer": reference_answer,
        "crag_evaluation": crag_eval,
        "standard_evaluation": standard_eval,
        "comparison": comparison
    }

In [38]:
def compare_responses(query, crag_response, standard_response, reference_answer=None):
    """
    CRAG 응답과 일반 RAG 응답을 비교 평가합니다.

    Args:
        query (str): 사용자 질문
        crag_response (str): CRAG 방식의 응답
        standard_response (str): 일반 RAG 방식의 응답
        reference_answer (str, optional): 기준 정답 (비교용, 선택)

    Returns:
        str: 비교 분석 결과 (자연어 분석)
    """
    # 시스템 프롬프트: 두 방식 비교 기준 안내
    system_prompt = """
    당신은 응답 생성 시스템을 비교 평가하는 전문가입니다.
    
    두 시스템을 비교하세요:

    1. CRAG (Corrective RAG): 문서의 관련도를 평가하고 필요 시 웹 검색을 병행하는 방식
    2. Standard RAG: 임베딩 유사도 기반으로 문서를 직접 검색하여 응답을 생성하는 방식

    아래 기준을 중심으로 두 응답을 비교 평가하세요:
    - 정보의 정확성 및 사실 기반 여부
    - 질문과의 관련성
    - 답변의 완전성 (질문의 모든 측면을 다루는지)
    - 명확성과 구성의 논리성
    - 출처 표기의 신뢰성과 구체성

    이 쿼리에 대해 어느 방식이 더 나은 응답을 제공했는지, 그 이유와 함께 설명해 주세요.
    """

    # 사용자 프롬프트 구성
    user_prompt = f"""
    질문: {query}

    CRAG 응답:
    {crag_response}

    Standard RAG 응답:
    {standard_response}
    """

    # 기준 정답이 있을 경우 포함
    if reference_answer:
        user_prompt += f"""
    기준 정답:
    {reference_answer}
    """

    try:
        # GPT-4 모델을 사용해 비교 분석 요청
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )

        # 분석 결과 반환
        return response.choices[0].message.content.strip()

    except Exception as e:
        # 오류 발생 시 메시지 반환
        print(f"응답 비교 중 오류 발생: {e}")
        return f"응답 비교 중 오류 발생: {str(e)}"

## Complete Evaluation Pipeline

In [39]:
def run_crag_evaluation(pdf_path, test_queries, reference_answers=None):
    """
    여러 개의 테스트 쿼리에 대해 CRAG 평가를 실행합니다.

    Args:
        pdf_path (str): 평가할 PDF 문서의 경로
        test_queries (List[str]): 평가에 사용할 쿼리 목록
        reference_answers (List[str], optional): 기준 정답 목록 (선택)

    Returns:
        Dict: 전체 평가 결과 (개별 결과 + 종합 분석 포함)
    """
    # 문서를 처리하고 벡터 스토어 생성
    vector_store = process_document(pdf_path)

    results = []

    # 쿼리별로 CRAG vs Standard RAG 평가 수행
    for i, query in enumerate(test_queries):
        print(f"\n\n***쿼리 {i+1}/{len(test_queries)} 평가 중***")
        print(f"질문: {query}")

        # 기준 정답이 있는 경우
        reference = None
        if reference_answers and i < len(reference_answers):
            reference = reference_answers[i]

        # CRAG vs 일반 RAG 비교 실행
        result = compare_crag_vs_standard_rag(query, vector_store, reference)
        results.append(result)

        # 비교 결과 출력
        print("\n***비교 결과***")
        print(result["comparison"])

    # 전체 평가 결과 요약 분석 생성
    overall_analysis = generate_overall_analysis(results)

    return {
        "results": results,  # 개별 쿼리별 평가 결과
        "overall_analysis": overall_analysis  # 전체 요약 분석
    }

In [40]:
def generate_overall_analysis(results):
    """
    여러 쿼리 평가 결과를 종합하여 CRAG vs Standard RAG의 전체 성능을 분석합니다.

    Args:
        results (List[Dict]): 각 쿼리별 평가 결과 리스트

    Returns:
        str: 두 접근 방식의 전반적인 성능 분석 결과 (자연어 설명)
    """
    # 시스템 프롬프트: 분석 방향과 항목 정의
    system_prompt = """
    당신은 정보 검색 및 응답 생성 시스템의 성능을 분석하는 전문가입니다.
    다수의 테스트 쿼리 결과를 바탕으로 CRAG(Corrective RAG)와 Standard RAG의 성능을 비교 분석하세요.

    다음 항목에 초점을 맞추어 분석하세요:
    1. CRAG가 더 잘 작동한 상황과 그 이유
    2. Standard RAG가 더 나았던 경우와 그 이유
    3. 각 방식의 전반적인 강점과 약점
    4. 각 방식을 사용하기 적합한 상황에 대한 실용적인 권장 사항
    """

    # 각 쿼리의 요약 결과 텍스트로 정리
    evaluations_summary = ""
    for i, result in enumerate(results):
        evaluations_summary += f"쿼리 {i+1}: {result['query']}\n"
        if 'crag_evaluation' in result and 'overall_score' in result['crag_evaluation']:
            crag_score = result['crag_evaluation'].get('overall_score', 'N/A')
            evaluations_summary += f"- CRAG 점수: {crag_score}\n"
        if 'standard_evaluation' in result and 'overall_score' in result['standard_evaluation']:
            std_score = result['standard_evaluation'].get('overall_score', 'N/A')
            evaluations_summary += f"- Standard RAG 점수: {std_score}\n"
        evaluations_summary += f"- 비교 요약: {result['comparison'][:200]}...\n\n"

    # 사용자 프롬프트: 실제 평가 데이터 기반 분석 요청
    user_prompt = f"""
    다음은 총 {len(results)}개의 쿼리에 대해 CRAG vs Standard RAG 방식을 비교한 평가 요약입니다.

    {evaluations_summary}

    이 정보를 바탕으로 두 방식의 상대적인 강점과 약점을 중심으로 종합적인 분석을 작성해 주세요.
    특히 어떤 조건에서 어느 방식이 더 효과적인지를 명확히 설명해 주세요.
    """

    try:
        # GPT-4를 이용해 전체 분석 생성
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0  # 객관적인 분석을 위한 낮은 창의성 설정
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"전체 분석 생성 중 오류 발생: {e}")
        return f"전체 분석 생성 중 오류 발생: {str(e)}"

## Evaluation of CRAG with Test Queries

In [41]:
# AI 정보가 담긴 PDF 문서 경로
pdf_path = "dataset/AI_Understanding.pdf"

# AI 관련 쿼리를 포함한 전체 평가 실행
test_queries = [
    "머신러닝은 전통적인 프로그래밍과 어떻게 다른가요?",
]

# 평가 품질 향상을 위한 (선택적) 기준 정답
reference_answers = [
    "머신러닝은 컴퓨터가 명시적인 규칙 없이 데이터를 통해 패턴을 학습한다는 점에서 전통적인 프로그래밍과 다릅니다. 전통적인 프로그래밍에서는 개발자가 일일이 규칙을 작성하지만, 머신러닝에서는 알고리즘이 데이터에서 규칙을 스스로 학습합니다.",
]

# CRAG vs Standard RAG 전체 비교 평가 실행
evaluation_results = run_crag_evaluation(pdf_path, test_queries, reference_answers)

# 전체 평가 분석 결과 출력
print("\n***CRAG vs Standard RAG 전체 성능 분석***")
print(evaluation_results["overall_analysis"])

dataset/AI_Understanding.pdf에서 텍스트 추출 중...
총 21개의 텍스트 청크가 생성되었습니다.
청크 임베딩 생성 중...
21개의 청크로 구성된 벡터 저장소가 생성되었습니다.


***쿼리 1/1 평가 중***
질문: 머신러닝은 전통적인 프로그래밍과 어떻게 다른가요?

***CRAG 실행 중***

=== CRAG 방식으로 쿼리 처리 중: 머신러닝은 전통적인 프로그래밍과 어떻게 다른가요? ===

초기 문서 검색 중...
문서 관련도 평가 중...
문서 관련도 점수: 0.90
문서 관련도 점수: 0.70
문서 관련도 점수: 0.20
높은 관련도 (0.90) - 문서 직접 활용
최종 응답 생성 중...

***일반 RAG 실행 중***

***CRAG 응답 평가 중***

***일반 RAG 응답 평가 중***

***두 방식 응답 비교 중***

***비교 결과***
두 시스템의 응답을 비교 평가해보겠습니다.

### 1. 정보의 정확성 및 사실 기반 여부
- **CRAG 응답**: 머신러닝과 전통적인 프로그래밍의 차이를 잘 설명하고 있으며, 각 개념에 대한 예시도 적절합니다. 그러나 "내부 문서"라는 출처는 신뢰성이 떨어질 수 있습니다.
- **Standard RAG 응답**: 정보의 정확성이 높고, 머신러닝의 세 가지 유형(지도 학습, 비지도 학습, 강화 학습)을 명확히 구분하여 설명합니다. "문서"라는 출처도 다소 모호하지만, CRAG보다 더 일반적인 출처로 보입니다.

### 2. 질문과의 관련성
- 두 응답 모두 질문에 대한 관련성을 잘 유지하고 있습니다. 머신러닝과 전통적인 프로그래밍의 차이를 명확히 설명하고 있습니다.

### 3. 답변의 완전성
- **CRAG 응답**: 머신러닝의 다양한 방법론(지도 학습, 비지도 학습, 강화 학습)을 언급하였으나, 각 방법론에 대한 구체적인 예시가 부족합니다.
- **Standard RAG 응답**: 머신러닝의 세 가지 유형을 명확히 나누어 설명하고, 각 유형에 대한 예시를 제공하여 답변